<a href="https://colab.research.google.com/github/elk-cloner/RL/blob/master/CartPole_Simple_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# using https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t guide to
# show openai gym env in jupyter
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip uninstall pyglet
!pip install pyglet==1.3.2
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install tensorflow-gpu

Uninstalling pyglet-1.3.2:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/pyglet-1.3.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/pyglet/*
Proceed (y/n)? y
  Successfully uninstalled pyglet-1.3.2
  Using cached https://files.pythonhosted.org/packages/1c/fc/dad5eaaab68f0c21e2f906a94ddb98175662cc5a654eee404d59554ce0fa/pyglet-1.3.2-py2.py3-none-any.whl
Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.4.0)
     |████████████████████████████████| 380.8MB 70kB/s 
     |████████████████████████████████| 3.8MB 40.7MB/s 
     |████████████████████████████████| 450kB 50.9MB/s 
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
     

In [0]:
import io
import gym
import math
import glob
import base64
import random
import matplotlib
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from itertools import count
from collections import deque
from IPython.display import HTML
from gym.wrappers import Monitor
from gym import logger as gymlogger
gymlogger.set_level(40) #error only
from IPython import display as ipythondisplay

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [3]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# env = wrap_env(gym.make("MountainCar-v0"))
# show_video()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
# define fome global variables
MEMORY_BUFFER = deque(maxlen=10000)
BATCH_SIZE = 32
GAMMA = 0.999
ORACLE_UPDATE_INTERVAL = 10
ACTION_SPACE = None

In [0]:
# define very simple DQN network
def DQNModel():
    policy_model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(input_shape=(4,), units=16, activation="relu"),
        tf.keras.layers.Dense(units=16, activation="relu"),
        tf.keras.layers.Dense(units=2, activation="linear")
    ])
    return policy_model

In [14]:
# define Q-network and Q*-network(oracle)
dqn_net = DQNModel()
oracle_net = tf.keras.models.clone_model(dqn_net)

# define loss function and optimizer
loss_func = tf.keras.losses.MeanSquaredError()
optimizer = optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=1)

print(dqn_net.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 16)                80        
_________________________________________________________________
dense_16 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 34        
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# using vanila epsilon greedy to choose action
def select_action(current_state):
    if np.random.rand() < 0.5:
        return random.sample(ACTION_SPACE,1)[0]
    current_state = np.array(current_state).reshape(1, -1)
    action_score = dqn_net.predict(current_state)
    return tf.math.argmax(action_score, axis=1).numpy()[0]

In [0]:
def optimize_model():
    if len(MEMORY_BUFFER) < BATCH_SIZE:
        return 0

    sample_batch = random.sample(MEMORY_BUFFER, BATCH_SIZE)
    y = np.zeros(shape=[1, len(sample_batch)])

    for i, sample in enumerate(sample_batch):
        current_state, action, next_state, reward, done = sample
        if done:
            y[0][i] = reward
        else:
            next_state = np.array(next_state).reshape(1, -1)
            target_model_score = tf.math.reduce_max(oracle_net.predict(next_state)).numpy()
            y[0][i] = GAMMA * target_model_score + reward

    batch_current_state = []
    batch_action = np.zeros(shape=[BATCH_SIZE, 2], dtype=np.int32)
    for i, sample in enumerate(sample_batch):
        current_state, action, next_state, reward, _ = sample
        batch_current_state.append(current_state)
        batch_action[i][0] = i
        batch_action[i][1] = action

    batch_current_state = np.array(batch_current_state)

    with tf.GradientTape() as gradient_tape:
        x = dqn_net(batch_current_state)
        x = tf.gather_nd(x, batch_action)
        loss = loss_func(y, x)
        gradient = gradient_tape.gradient(loss, dqn_net.trainable_variables)
        optimizer.apply_gradients(zip(gradient, dqn_net.trainable_variables))

    return 1

In [0]:
def solved():
    # Considered solved when the average reward is greater than or equal to 195.0 over 100 consecutive trials.
    oracle_net.load_weights("./pre_train_model/")
    env = gym.make("CartPole-v0")
    trials_rewards = []
    for i in range(100):
        current_state = env.reset()
        total_rewards = 0
        while True:
            current_state = np.array(current_state).reshape(1, -1)
            action_score = oracle_net.predict(current_state)
            action = tf.math.argmax(action_score, axis=1).numpy()[0]
            next_state, reward, done, info = env.step(action)
            current_state = next_state
            if done:
                break
            total_rewards += reward
        trials_rewards.append(total_rewards)
    mean_rewards = sum(trials_rewards) / len(trials_rewards)
    print(f"mean rewards for 100 consecutive trials: {mean_rewards}") 
    if mean_rewards >= 195.0:
        return True
    return False

In [0]:
def render_model():
    oracle_net.load_weights("./pre_train_model/")
    env = wrap_env(gym.make("CartPole-v0"))
    current_state = env.reset()
    while True:
        env.render()
        current_state = np.array(current_state).reshape(1, -1)
        action_score = oracle_net.predict(current_state)
        action = tf.math.argmax(action_score, axis=1).numpy()[0]
        next_state, reward, done, info = env.step(action)
        current_state = next_state
        if done:
            break
    env.close()
    show_video()

In [75]:
score_epoch = []
optimize_steps = 0
env = gym.make("CartPole-v0")
ACTION_SPACE = list(range(env.action_space.n))

for trial_number in count():
    current_state = env.reset()
    print(f"trial number: {trial_number}")
    while True:
        action_num = select_action(current_state)
        next_state, reward, done, info = env.step(action_num)
        reward = reward if not done else -1 * reward
        MEMORY_BUFFER.append(tuple([current_state, action_num, next_state, reward, done]))
        optimize_steps += optimize_model()
        current_state = next_state
        if optimize_steps and optimize_steps % ORACLE_UPDATE_INTERVAL == 0:
            oracle_net.set_weights(dqn_net.get_weights())
            oracle_net.save_weights("./pre_train_model/")
        if done:
            break
    if optimize_steps > 0 and solved():
        print(f"solved after {trial_number} episods")
        break

trial number: 0
mean rewards for 100 consecutive trials: 8.39
trial number: 1
mean rewards for 100 consecutive trials: 8.27
trial number: 2
mean rewards for 100 consecutive trials: 8.48
trial number: 3
mean rewards for 100 consecutive trials: 8.31
trial number: 4
mean rewards for 100 consecutive trials: 8.4
trial number: 5
mean rewards for 100 consecutive trials: 8.5
trial number: 6
mean rewards for 100 consecutive trials: 8.27
trial number: 7
mean rewards for 100 consecutive trials: 8.43
trial number: 8
mean rewards for 100 consecutive trials: 8.35
trial number: 9
mean rewards for 100 consecutive trials: 8.41
trial number: 10
mean rewards for 100 consecutive trials: 8.29
trial number: 11
mean rewards for 100 consecutive trials: 8.44
trial number: 12
mean rewards for 100 consecutive trials: 8.32
trial number: 13
mean rewards for 100 consecutive trials: 8.31
trial number: 14
mean rewards for 100 consecutive trials: 8.32
trial number: 15
mean rewards for 100 consecutive trials: 8.37
tria

In [83]:
render_model()